In [ ]:
# setting the Thesis project folder as working directory
%cd "../.."

# Losses

In [ ]:
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import os
from datetime import datetime
import tensorflow.keras.layers as tfkl
import tensorflow.keras as tfk 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate

from sklearn.model_selection import train_test_split
import math
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.losses as tfkloss
from sklearn.metrics import recall_score, precision_score
import tensorflow.math as tfm

## Setting default parameters

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# plots sizes
plt.rcParams['figure.figsize'] = [9, 9]

In [ ]:
import tensorflow.keras.backend as K

def my_coeff(y_true, y_pred, const=K.epsilon()):

    # flatten 2-d tensors
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    # get true pos (TP), false neg (FN), false pos (FP).
    tp  = tf.reduce_sum(y_true * y_pred)
    tn = tf.reduce_sum((1 - y_true) * (1 - y_pred))
    fn = tf.reduce_sum(y_true * (1-y_pred))
    fp = tf.reduce_sum((1-y_true) * y_pred)

    recall = tp/(tf.math.add(tp, fn))
    precision = tp / tf.math.add(tp, fp)

    coeff = tf.math.add(recall*0.6,  precision*0.4)

    return coeff

def my_loss(y_true, y_pred, const=K.epsilon()):

    # tf tensor casting
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    y_pred = tf.squeeze(y_pred)
    y_true = tf.squeeze(y_true)

    loss_val = 1 - my_coeff(y_true, y_pred, const=const)

    return loss_val

In [ ]:
def my_precision(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true==y_pred, y_pred==1))
    tn = np.sum(np.logical_and(y_true==y_pred, y_pred==0))
    fp  = np.sum(np.logical_and(y_true!=y_pred, y_pred==1))
    fn = np.sum(np.logical_and(y_true!=y_pred, y_pred==0))

    return tp/(tp + fp)

def my_recall(y_true, y_pred):
    tp = np.sum(np.logical_and(y_true==y_pred, y_pred==1))
    tn = np.sum(np.logical_and(y_true==y_pred, y_pred==0))
    fp  = np.sum(np.logical_and(y_true!=y_pred, y_pred==1))
    fn = np.sum(np.logical_and(y_true!=y_pred, y_pred==0))

    return tp/(tp + fn)

def my_metric(y_true, y_pred):
    recall = my_recall(y_true, y_pred)
    precision = my_precision(y_true,y_pred)
    loss =  1 - (recall*0.6 + precision*0.4)

    return loss

In [ ]:
true = np.array([[1, 1, 1, 1,],       
                 [0, 0, 1, 1,],
                 [0, 1, 1, 1,],
                 [1, 0, 0, 0,],])

pred = np.array([[1, 1, 1, 1,],       
                 [0, 0, 1, 1,],
                 [0, 1, 1, 1,],
                 [0, 0, 0, 0,],])


my_recall(true, pred), my_precision(true, pred),  my_metric(true, pred)

In [ ]:
my_loss(true, pred)

In [ ]:
tf.math.reduce_sum(tf.where(tf.math.logical_and(true==pred, pred==1), 1, 0))

In [ ]:
tf.math.reduce_sum(tf.where(tf.math.logical_and(true==pred, pred==1), 1, 0))